## Sentinel 3 Data Streaming

This service offers compressed Sentinel 3 NDVI data and makes it available via a high quality and memory efficient streaming solution.


## Prerequisites
### DestinE Platform Credentials

You need to have an account on the [Destination Earth Platform](https://auth.destine.eu/realms/desp/account).

#### ⚠️ Warning: Authorized Access Only
The usage of this notebook and data access is reserved only to authorized user groups.

## Access the Data
With a DESP account you can access the stream data proposed in this tutorial.

In [ ]:
%%capture cap
%run ./auth.py

In [ ]:
output = cap.stdout.split('\n')
refresh_token = output[1]
token = output[2]
print(token)

# Imports and general definitions
We start by importing necessary packages and definitions regarding the resolution and the endpoint to the streaming api.

Note: The API token must be set here including the user group. This happens in **Authentification**.

In [ ]:
from datetime import datetime
import xarray as xa
import rioxarray  # noqa
from rasterio.transform import from_bounds
from dtelib_s3 import DTEStreamer_S3
import gc
import matplotlib.pyplot as plt
from IPython import display
import numpy as np

FORMAT = '%Y-%m-%dT%H:%M'

# Parameters for stream access

Here the parameters are set to access the data from the service.

*variable*: an abbreviated variable for the data</br>
*program_subset*: a name for the program subest for the data</br>
*start_date*: the time and date to start the stream</br>
*end_date*: the time and date to end the stream</br>
</br>

To select a stream, chose parameter values from the table above, or if you have a *code snippet*, use it to replace the code in the cell below:

In [ ]:
continent = "Africa"

start_date = datetime.strptime("2024-01-01T00:00", FORMAT)
end_date = datetime.strptime("2024-12-31T00:00", FORMAT)

# Defining the Area

We pick the Okawango Delta in Botswana to showcase the NDVI over one year.

In [ ]:
south_africa = [
    {
        'type': 'Polygon',
        'coordinates':     [[
            [
              22.736787239715994,
              -19.104234073492805
            ],
            [
              22.736787239715994,
              -20.087634051040524
            ],
            [
              23.898375761799315,
              -20.087634051040524
            ],
            [
              23.898375761799315,
              -19.104234073492805
            ],
            [
              22.736787239715994,
              -19.104234073492805
            ]
        ]]
    }
]



# Loading and plotting the data

With a DTEStreamer_S3 object the data from Africa is loaded. No-Data values are marked as NAN and the data is georeferenced, to cut down to the AOI of the Okawango Delta. The Average over the area is calculated, stored in a list and the plot updated.

In [ ]:
streamer = DTEStreamer_S3(continent=continent, start_date=start_date, end_date=end_date, token=token)

# Change this to values from 1 to 24 to affect in the plotting rate.
# The final plot is the same for any setting.
plot_every_x_hours = 4

# initialize the plot figure
fig, ax = plt.subplots(1, 1, figsize=(15, 6))
hdisplay = display.display('', display_id=True)

ax.set_xlabel(f'satellite images')
ax.set_ylabel(f'NDVI')
plt.title(f'Average NDVI from S3A of the Okawango Delta between {start_date.strftime("%Y-%m-%d")} and {end_date.strftime("%Y-%m-%d")}')

data, t = [], []
ax.plot(data,t)
line = ax.lines[0]

for i, (image, metadata) in enumerate(streamer.load_next_image()):

    # only pick S3A to avoid duplicates
    if metadata["satellite"] == "S3B":
        continue
        
    # set no data values
    image[image < 0.05] = np.nan

    # georeference data
    da = xa.DataArray(image,
                      dims=['latitude', 'longitude'],
                      name="NDVI"
                     )

    transform = from_bounds(*streamer.bbox(), streamer.nx(), streamer.ny())

    da = da.rio.write_crs("EPSG:4326")
    da = da.rio.write_transform(transform)
    
    # isolate South African data
    da = da.rio.clip(geometries=south_africa, drop=True)
      
    data.append(np.average(da.mean(skipna=True).item()))
    t.append(i)

    line.set_xdata(t)
    line.set_ydata(data)
        
    # scale plot to fit temperatures
    ax.set_ylim(-0.08, 1)
    ax.set_xlim(min(t)-1, max(t)+1)

    hdisplay.update(fig)
    
plt.close(fig)
print("done")